In [5]:
# To support both python 2 and python 3
from __future__ import division, print_function, unicode_literals

# Common imports
import numpy as np
import os

# to make this notebook's output stable across runs
def reset_graph(seed=42):
    tf.reset_default_graph()
    tf.set_random_seed(seed)
    np.random.seed(seed)

# To plot pretty figures
%matplotlib inline
import matplotlib
import matplotlib.pyplot as plt
plt.rcParams['axes.labelsize'] = 14
plt.rcParams['xtick.labelsize'] = 12
plt.rcParams['ytick.labelsize'] = 12

# Where to save the figures
PROJECT_ROOT_DIR = "."
CHAPTER_ID = "ann"

def save_fig(fig_id, tight_layout=True):
    path = os.path.join(PROJECT_ROOT_DIR, "images", CHAPTER_ID, fig_id + ".png")
    print("Saving figure", fig_id)
    if tight_layout:
        plt.tight_layout()
    plt.savefig(path, format='png', dpi=300)

In [6]:
import sys
sys.path.append('../../../')

from tensorflow_models import DNNClassifier, CNNClassifier, GridSearchCNN, GridSearchDNN, RandomSearchCNN

In [7]:
processedDataX_path = "../../../preprocessedSamples_X_samples_allGuitar_20480_Mm7_R1D.data"
processedDatay_path = "../../../preprocessedSamples_y_samples_allGuitar_20480_Mm7_R1D.data"
processedData_path = ""

if os.path.isfile(processedDataX_path): #if already preprocessed
    processedX = np.load(processedDataX_path)
    processedy = np.load(processedDatay_path)
else:
    import pandas as pd
    import librosa
    df = pd.read_csv('samples_allGuitar_20480_Mm7_R1D.csv')
    # df = pd.read_csv('samples_nylonGuitar_1024_Mm7_R03.csv')
    # df = pd.read_csv('../CachedData.csv')

    X_load = np.array(df.iloc[:,:-1], dtype=np.float)
    y_load = np.array(df.iloc[:,-1], dtype=np.float)
    processedX = np.zeros((len(X_load),12,80,1), dtype=np.float)
    processedy = np.zeros(len(y_load), dtype=np.float)
    for i in range(len(X_load)):
        # sample = librosa.core.stft(y=X_load[i], n_fft=2048, win_length=128, window='hamming', center=True, dtype=np.float32, pad_mode='reflect')
        sample = librosa.feature.chroma_stft(y=X_load[i], sr=44100, n_fft=20480, hop_length=258)
        sample = np.atleast_3d(sample)
        processedX[i] = sample
        processedy[i] = y_load[i]
        if i % 400 == 0:
            print(i)

    processedX.dump(processedDataX_path)
    processedy.dump(processedDatay_path)

from sklearn.utils import shuffle
print(processedy)
sprocessedX, sprocessedy = shuffle(processedX, processedy)
print(len(sprocessedX))

trainRange = int(len(sprocessedX) * 0.8)
validRange = int(len(sprocessedX) * 9)
testRange = int(len(sprocessedX) * 0.1)


X_train = np.array(sprocessedX[:trainRange], dtype=np.float)
y_train = np.array(sprocessedy[:trainRange], dtype=np.float)

X_valid = np.array(sprocessedX[trainRange:validRange], dtype=np.float)
y_valid = np.array(sprocessedy[trainRange:validRange], dtype=np.float)

X_test = np.array(sprocessedX[testRange:], dtype=np.float)
y_test = np.array(sprocessedy[testRange:], dtype=np.float)
print(y_test[1])
print(sprocessedX.shape)

[  0.   0.   0. ...,  45.  45.  45.]
3214
28.0
(3214, 12, 80, 1)


In [8]:
chords = np.zeros(50)
for chord in range(50):
    for sample in sprocessedy:
        if sample == chord:
            chords[chord] += 1

print(chords)

[  82.   68.   64.   68.   58.   57.   56.   67.   76.  108.   58.   60.
   46.   57.   55.   57.   72.   66.   69.   70.   76.   68.   60.   62.
   71.   49.   61.   62.  101.  107.   75.   47.  103.   72.   57.   67.
   74.   63.   73.   46.   43.   77.   54.   58.   62.   73.   72.   67.
    0.    0.]


In [9]:
import tensorflow as tf

def leaky_relu(alpha=0.01):
		def parametrized_leaky_relu(z, name=None):
			return tf.maximum(alpha * z, z, name=name)
		return parametrized_leaky_relu


""""
test 01:
params = {'n_hidden_layers': [1],
	'n_neurons' : [500, 300],
	'optimizer_class' : [tf.train.AdagradOptimizer, tf.train.AdamOptimizer],
	'learning_rate' : [0.01],
	'batch_size' : [100],
	'activation' : [leaky_relu(), tf.nn.elu],
	'dropout_rate' : [0.1, 0.000001, 0.2],
	'conv1' : [{'conv1_fmaps':16, 'conv1_ksize':5, 'conv1_stride':1, 'conv1_dropout':0.1, 'conv1_activation':tf.nn.elu},
               {'conv1_fmaps':16, 'conv1_ksize':5, 'conv1_stride':1, 'conv1_dropout':0.1, 'conv1_activation':tf.nn.relu}],
	'conv2' : [{'conv2_fmaps':32, 'conv2_ksize':5, 'conv2_stride':1, 'conv2_dropout':0.1, 'conv2_activation':tf.nn.relu},
              {'conv2_fmaps':32, 'conv2_ksize':5, 'conv2_stride':1, 'conv2_dropout':0.1, 'conv2_activation':tf.nn.elu}],
    'architecture' : [1, 2]
}

winner:

 {'n_hidden_layers': 1, 'n_neurons': 300, 'optimizer_class': <class 'tensorflow.python.training.adam.AdamOptimizer'>, 'learning_rate': 0.01, 'batch_size': 100, 'activation': <function leaky_relu.<locals>.parametrized_leaky_relu at 0x0000020C8986DA60>, 'dropout_rate': 0.1, 'conv1': {'conv1_fmaps': 16, 'conv1_ksize': 5, 'conv1_stride': 1, 'conv1_dropout': 0.1, 'conv1_activation': <function relu at 0x0000020C86690620>}, 'conv2': {'conv2_fmaps': 32, 'conv2_ksize': 5, 'conv2_stride': 1, 'conv2_dropout': 0.1, 'conv2_activation': <function relu at 0x0000020C86690620>}, 'architecture': 1, 'accuracy_rate': 0.87231835126876833}\n",
 

test 02:



params = {'n_hidden_layers': [1],
	'n_neurons' : [200, 300],
	'optimizer_class' : [tf.train.AdagradOptimizer, tf.train.AdamOptimizer],
	'learning_rate' : [0.01, 0.05],
	'batch_size' : [100, 200],
	'activation' : [leaky_relu(), tf.nn.elu],
	'dropout_rate' : [0.1, 0.000001, 0.2],
	'conv1' : [{'conv1_fmaps':16, 'conv1_ksize':5, 'conv1_stride':1, 'conv1_dropout':0.1, 'conv1_activation':tf.nn.relu}],
	'conv2' : [{'conv2_fmaps':32, 'conv2_ksize':5, 'conv2_stride':1, 'conv2_dropout':0.1, 'conv2_activation':tf.nn.relu}],
    'architecture' : [1, 2]
}

Winner:
"{'n_hidden_layers': 1, 'n_neurons': 300, 'optimizer_class': <class 'tensorflow.python.training.adam.AdamOptimizer'>, 'learning_rate': 0.01, 'batch_size': 100, 'activation': <function leaky_relu.<locals>.parametrized_leaky_relu at 0x0000015A746872F0>, 'dropout_rate': 0.1, 'conv1': {'conv1_fmaps': 16, 'conv1_ksize': 5, 'conv1_stride': 1, 'conv1_dropout': 0.1, 'conv1_activation': <function relu at 0x0000015A70EB52F0>}, 'conv2': {'conv2_fmaps': 32, 'conv2_ksize': 5, 'conv2_stride': 1, 'conv2_dropout': 0.1, 'conv2_activation': <function relu at 0x0000015A70EB52F0>}, 'architecture': 1, 'accuracy_rate': 0.87543249130249023}\n",


params = {'n_hidden_layers': [0, 1, 2],
	'n_neurons' : [250, 300],
	'optimizer_class' : [tf.train.AdamOptimizer],
	'learning_rate' : [0.01, 0.05],
	'batch_size' : [100, 200],
	'activation' : [leaky_relu()],
	'dropout_rate' : [0.1, 0.000001, 0.2, 0.3],
	'conv1' : [{'conv1_fmaps':16, 'conv1_ksize':5, 'conv1_stride':1, 'conv1_dropout':0.1, 'conv1_activation':tf.nn.relu},
               {'conv1_fmaps':16, 'conv1_ksize':5, 'conv1_stride':1, 'conv1_dropout':0.2, 'conv1_activation':tf.nn.relu}],
	'conv2' : [{'conv2_fmaps':32, 'conv2_ksize':5, 'conv2_stride':1, 'conv2_dropout':0.1, 'conv2_activation':tf.nn.relu},
               {'conv2_fmaps':32, 'conv2_ksize':5, 'conv2_stride':1, 'conv2_dropout':0.2, 'conv2_activation':tf.nn.relu}],
    'architecture' : [1]
}

Winner:
"{'n_hidden_layers': 2, 'n_neurons': 300, 'optimizer_class': <class 'tensorflow.python.training.adam.AdamOptimizer'>, 'learning_rate': 0.05, 'batch_size': 200, 'activation': <function leaky_relu.<locals>.parametrized_leaky_relu at 0x0000015A74687268>, 'dropout_rate': 0.3, 'conv1': {'conv1_fmaps': 16, 'conv1_ksize': 5, 'conv1_stride': 1, 'conv1_dropout': 0.1, 'conv1_activation': <function relu at 0x0000015A70EB52F0>}, 'conv2': {'conv2_fmaps': 32, 'conv2_ksize': 5, 'conv2_stride': 1, 'conv2_dropout': 0.2, 'conv2_activation': <function relu at 0x0000015A70EB52F0>}, 'architecture': 1, 'accuracy_rate': 0.8802768468856812}\n",


"""""

params = {'n_hidden_layers': [2, 3],
	'n_neurons' : [250, 300, 350],
	'optimizer_class' : [tf.train.AdamOptimizer],
	'learning_rate' : [0.01, 0.05],
	'batch_size' : [200, 250],
	'activation' : [leaky_relu()],
	'dropout_rate' : [0.3, 0.4],
	'conv1' : [{'conv1_fmaps':16, 'conv1_ksize':5, 'conv1_stride':1, 'conv1_dropout':0.1, 'conv1_activation':tf.nn.relu}],
	'conv2' : [{'conv2_fmaps':32, 'conv2_ksize':5, 'conv2_stride':1, 'conv2_dropout':0.2, 'conv2_activation':tf.nn.relu}],
    'architecture' : [1]
}
grid_search = GridSearchCNN(params, k_fold=5)
grid_search.fit(X_train, y_train, X_valid, y_valid)

Trining CNN with parameters: n_hidden_layers: 2, n_neurons: 250, optimizer_class: <class 'tensorflow.python.training.adam.AdamOptimizer'>, learning_rate: 0, batch_size: 200, activation: <function leaky_relu.<locals>.parametrized_leaky_relu at 0x0000015A74687048>, dropout_rate: 0, architecture: 1, conv1: {'conv1_fmaps': 16, 'conv1_ksize': 5, 'conv1_stride': 1, 'conv1_dropout': 0.1, 'conv1_activation': <function relu at 0x0000015A70EB52F0>}, conv2: {'conv2_fmaps': 32, 'conv2_ksize': 5, 'conv2_stride': 1, 'conv2_dropout': 0.2, 'conv2_activation': <function relu at 0x0000015A70EB52F0>} .
Training and testing fold 0
ranges:
2312 2890
shapes:
(2314, 12, 80, 1) (578, 12, 80, 1)
(2314,) (578,)
Epoch 0, train accuracy: 65.0000%, valid. accuracy: 58.3851%, valid. best loss: 1.586686
Epoch 1, train accuracy: 75.0000%, valid. accuracy: 70.8075%, valid. best loss: 1.048549
Epoch 2, train accuracy: 75.0000%, valid. accuracy: 75.1553%, valid. best loss: 0.781454
Epoch 3, train accuracy: 80.0000%, val

Epoch 4, train accuracy: 90.0000%, valid. accuracy: 80.7453%, valid. best loss: 0.617943
Epoch 5, train accuracy: 85.0000%, valid. accuracy: 81.3665%, valid. best loss: 0.603096
Epoch 6, train accuracy: 95.0000%, valid. accuracy: 85.7143%, valid. best loss: 0.585737
Epoch 7, train accuracy: 95.0000%, valid. accuracy: 84.7826%, valid. best loss: 0.513686
Epoch 8, train accuracy: 100.0000%, valid. accuracy: 83.5404%, valid. best loss: 0.513686
Epoch 9, train accuracy: 100.0000%, valid. accuracy: 85.7143%, valid. best loss: 0.513686
Epoch 10, train accuracy: 95.0000%, valid. accuracy: 81.9876%, valid. best loss: 0.513686
Epoch 11, train accuracy: 90.0000%, valid. accuracy: 83.5404%, valid. best loss: 0.513686
Epoch 12, train accuracy: 100.0000%, valid. accuracy: 85.7143%, valid. best loss: 0.513686
Epoch 13, train accuracy: 95.0000%, valid. accuracy: 85.4037%, valid. best loss: 0.513686
Epoch 14, train accuracy: 100.0000%, valid. accuracy: 86.6460%, valid. best loss: 0.513686
Epoch 15, tr

Epoch 1, train accuracy: 75.0000%, valid. accuracy: 67.3913%, valid. best loss: 0.958878
Epoch 2, train accuracy: 90.0000%, valid. accuracy: 75.7764%, valid. best loss: 0.844428
Epoch 3, train accuracy: 85.0000%, valid. accuracy: 77.3292%, valid. best loss: 0.739974
Epoch 4, train accuracy: 85.0000%, valid. accuracy: 81.6770%, valid. best loss: 0.636449
Epoch 5, train accuracy: 85.0000%, valid. accuracy: 81.3665%, valid. best loss: 0.595858
Epoch 6, train accuracy: 95.0000%, valid. accuracy: 83.8509%, valid. best loss: 0.547401
Epoch 7, train accuracy: 95.0000%, valid. accuracy: 83.8509%, valid. best loss: 0.501987
Epoch 8, train accuracy: 100.0000%, valid. accuracy: 86.0249%, valid. best loss: 0.501987
Epoch 9, train accuracy: 95.0000%, valid. accuracy: 86.9565%, valid. best loss: 0.501987
Epoch 10, train accuracy: 100.0000%, valid. accuracy: 83.8509%, valid. best loss: 0.464394
Epoch 11, train accuracy: 100.0000%, valid. accuracy: 86.3354%, valid. best loss: 0.464394
Epoch 12, train 

Epoch 14, train accuracy: 100.0000%, valid. accuracy: 85.0932%, valid. best loss: 0.494656
Epoch 15, train accuracy: 100.0000%, valid. accuracy: 85.7143%, valid. best loss: 0.494656
Epoch 16, train accuracy: 100.0000%, valid. accuracy: 88.1988%, valid. best loss: 0.494656
Epoch 17, train accuracy: 100.0000%, valid. accuracy: 85.4037%, valid. best loss: 0.494656
Epoch 18, train accuracy: 100.0000%, valid. accuracy: 83.5404%, valid. best loss: 0.494656
Epoch 19, train accuracy: 100.0000%, valid. accuracy: 88.5093%, valid. best loss: 0.494656
Epoch 20, train accuracy: 100.0000%, valid. accuracy: 86.6460%, valid. best loss: 0.494656
Early stopping!
Final accuracy on test set: 0.858131
******************************************
{'n_hidden_layers': 2, 'n_neurons': 250, 'optimizer_class': <class 'tensorflow.python.training.adam.AdamOptimizer'>, 'learning_rate': 0.01, 'batch_size': 200, 'activation': <function leaky_relu.<locals>.parametrized_leaky_relu at 0x0000015A74687048>, 'dropout_rate': 

Epoch 20, train accuracy: 100.0000%, valid. accuracy: 86.3354%, valid. best loss: 0.516450
Epoch 21, train accuracy: 100.0000%, valid. accuracy: 87.5776%, valid. best loss: 0.516450
Early stopping!
Final accuracy on test set: 0.877163
Training and testing fold 3
ranges:
578 1156
shapes:
(2314, 12, 80, 1) (578, 12, 80, 1)
(2314,) (578,)
Epoch 0, train accuracy: 65.0000%, valid. accuracy: 57.4534%, valid. best loss: 1.500378
Epoch 1, train accuracy: 80.0000%, valid. accuracy: 72.9814%, valid. best loss: 0.975826
Epoch 2, train accuracy: 80.0000%, valid. accuracy: 78.2609%, valid. best loss: 0.758215
Epoch 3, train accuracy: 85.0000%, valid. accuracy: 77.9503%, valid. best loss: 0.691199
Epoch 4, train accuracy: 85.0000%, valid. accuracy: 80.7453%, valid. best loss: 0.588630
Epoch 5, train accuracy: 100.0000%, valid. accuracy: 81.9876%, valid. best loss: 0.588630
Epoch 6, train accuracy: 95.0000%, valid. accuracy: 82.2981%, valid. best loss: 0.565622
Epoch 7, train accuracy: 100.0000%, va

...

Epoch 0, train accuracy: 55.0000%, valid. accuracy: 51.8634%, valid. best loss: 1.587624
Epoch 1, train accuracy: 95.0000%, valid. accuracy: 67.0807%, valid. best loss: 1.101563
Epoch 2, train accuracy: 85.0000%, valid. accuracy: 77.0186%, valid. best loss: 0.838009
Epoch 3, train accuracy: 90.0000%, valid. accuracy: 77.6398%, valid. best loss: 0.636324
Epoch 4, train accuracy: 90.0000%, valid. accuracy: 80.4348%, valid. best loss: 0.636324
Epoch 5, train accuracy: 100.0000%, valid. accuracy: 82.6087%, valid. best loss: 0.603811
Epoch 6, train accuracy: 100.0000%, valid. accuracy: 80.4348%, valid. best loss: 0.573148
Epoch 7, train accuracy: 90.0000%, valid. accuracy: 82.9193%, valid. best loss: 0.516454
Epoch 8, train accuracy: 95.0000%, valid. accuracy: 84.4720%, valid. best loss: 0.516454
Epoch 9, train accuracy: 100.0000%, valid. accuracy: 80.4348%, valid. best loss: 0.510723
Epoch 10, train accuracy: 90.0000%, valid. accuracy: 84.1615%, valid. best loss: 0.510723
Epoch 11, train a

Epoch 8, train accuracy: 95.0000%, valid. accuracy: 81.0559%, valid. best loss: 0.505981
Epoch 9, train accuracy: 100.0000%, valid. accuracy: 86.6460%, valid. best loss: 0.505981
Epoch 10, train accuracy: 100.0000%, valid. accuracy: 82.6087%, valid. best loss: 0.500036
Epoch 11, train accuracy: 100.0000%, valid. accuracy: 83.8509%, valid. best loss: 0.500036
Epoch 12, train accuracy: 100.0000%, valid. accuracy: 83.5404%, valid. best loss: 0.500036
Epoch 13, train accuracy: 95.0000%, valid. accuracy: 85.7143%, valid. best loss: 0.500036
Epoch 14, train accuracy: 95.0000%, valid. accuracy: 86.3354%, valid. best loss: 0.500036
Epoch 15, train accuracy: 100.0000%, valid. accuracy: 84.1615%, valid. best loss: 0.500036
Epoch 16, train accuracy: 100.0000%, valid. accuracy: 82.2981%, valid. best loss: 0.500036
Epoch 17, train accuracy: 95.0000%, valid. accuracy: 89.1304%, valid. best loss: 0.500036
Epoch 18, train accuracy: 100.0000%, valid. accuracy: 83.5404%, valid. best loss: 0.500036
Epoch

Epoch 0, train accuracy: 75.0000%, valid. accuracy: 51.8634%, valid. best loss: 1.586260
Epoch 1, train accuracy: 90.0000%, valid. accuracy: 73.2919%, valid. best loss: 0.970332
Epoch 2, train accuracy: 85.0000%, valid. accuracy: 76.0870%, valid. best loss: 0.826418
Epoch 3, train accuracy: 90.0000%, valid. accuracy: 78.8820%, valid. best loss: 0.661818
Epoch 4, train accuracy: 70.0000%, valid. accuracy: 82.2981%, valid. best loss: 0.594449
Epoch 5, train accuracy: 90.0000%, valid. accuracy: 82.2981%, valid. best loss: 0.511460
Epoch 6, train accuracy: 70.0000%, valid. accuracy: 83.2298%, valid. best loss: 0.500150
Epoch 7, train accuracy: 95.0000%, valid. accuracy: 84.1615%, valid. best loss: 0.500150
Epoch 8, train accuracy: 95.0000%, valid. accuracy: 83.2298%, valid. best loss: 0.476191
Epoch 9, train accuracy: 95.0000%, valid. accuracy: 81.0559%, valid. best loss: 0.476191
Epoch 10, train accuracy: 90.0000%, valid. accuracy: 83.2298%, valid. best loss: 0.461340
Epoch 11, train accu

Epoch 11, train accuracy: 95.0000%, valid. accuracy: 83.2298%, valid. best loss: 0.505762
Epoch 12, train accuracy: 90.0000%, valid. accuracy: 83.2298%, valid. best loss: 0.505762
Epoch 13, train accuracy: 100.0000%, valid. accuracy: 82.9193%, valid. best loss: 0.505762
Epoch 14, train accuracy: 100.0000%, valid. accuracy: 85.4037%, valid. best loss: 0.505762
Epoch 15, train accuracy: 100.0000%, valid. accuracy: 87.2671%, valid. best loss: 0.505762
Epoch 16, train accuracy: 90.0000%, valid. accuracy: 85.4037%, valid. best loss: 0.505762
Epoch 17, train accuracy: 100.0000%, valid. accuracy: 84.7826%, valid. best loss: 0.505762
Epoch 18, train accuracy: 100.0000%, valid. accuracy: 87.2671%, valid. best loss: 0.505762
Epoch 19, train accuracy: 100.0000%, valid. accuracy: 83.2298%, valid. best loss: 0.505762
Epoch 20, train accuracy: 100.0000%, valid. accuracy: 84.7826%, valid. best loss: 0.505762
Early stopping!
Final accuracy on test set: 0.854671
****************************************

Epoch 12, train accuracy: 100.0000%, valid. accuracy: 84.4720%, valid. best loss: 0.507066
Epoch 13, train accuracy: 100.0000%, valid. accuracy: 79.1925%, valid. best loss: 0.507066
Epoch 14, train accuracy: 100.0000%, valid. accuracy: 86.0249%, valid. best loss: 0.486597
Epoch 15, train accuracy: 90.0000%, valid. accuracy: 81.3665%, valid. best loss: 0.486597
Epoch 16, train accuracy: 100.0000%, valid. accuracy: 84.1615%, valid. best loss: 0.486597
Epoch 17, train accuracy: 100.0000%, valid. accuracy: 85.7143%, valid. best loss: 0.486597
Epoch 18, train accuracy: 95.0000%, valid. accuracy: 83.8509%, valid. best loss: 0.486597
Epoch 19, train accuracy: 100.0000%, valid. accuracy: 85.0932%, valid. best loss: 0.486597
Epoch 20, train accuracy: 100.0000%, valid. accuracy: 83.8509%, valid. best loss: 0.486597
Epoch 21, train accuracy: 95.0000%, valid. accuracy: 85.0932%, valid. best loss: 0.486597
Epoch 22, train accuracy: 100.0000%, valid. accuracy: 83.5404%, valid. best loss: 0.486597
Ep

In [23]:
# global cnn
cnn = CNNClassifier(batch_size=50, learning_rate=0.01, n_hidden_layers=5, n_neurons=700, 
                    optimizer_class=tf.train.AdagradOptimizer, activation=leaky_relu(alpha=0.01), 
                    conv1={'conv1_fmaps':16, 'conv1_ksize':5, 'conv1_stride':1, 'conv1_dropout':0.2, 'conv1_activation':tf.nn.elu},
                   conv2={'conv2_fmaps':32, 'conv2_ksize':5, 'conv2_stride':1, 'conv2_dropout':0.4, 'conv2_activation':tf.nn.elu})
range1 = 0
range2 = 350
X_train2 = X_train[:range1]
X_train2 = np.vstack([X_train2, X_train[range2:]])
print(X_train2.shape)
y_train2 = y_train[:range1]
y_train2 = np.append(y_train2, y_train[range2:])

X_test2 = X_test[range1:range2]
y_test2 = y_test[range1:range2]

#cnn.fit(X_train=X_train[350:], y_train=y_train[350:], X_valid=X_valid, y_valid=y_valid)
cnn.fit(X_train=X_train2, y_train=y_train2, X_valid=None, y_valid=None)

score = cnn.accuracy_score(X_test2, y_test2)
score

(2542, 12, 80, 1)
Epoch 0, train accuracy: 58.0000%
Epoch 1, train accuracy: 66.0000%
Epoch 2, train accuracy: 64.0000%
Epoch 3, train accuracy: 76.0000%
Epoch 4, train accuracy: 80.0000%
Epoch 5, train accuracy: 84.0000%
Epoch 6, train accuracy: 64.0000%
Epoch 7, train accuracy: 76.0000%
Epoch 8, train accuracy: 86.0000%
Epoch 9, train accuracy: 64.0000%
Epoch 10, train accuracy: 72.0000%
Epoch 11, train accuracy: 78.0000%
Epoch 12, train accuracy: 84.0000%
Epoch 13, train accuracy: 64.0000%
Epoch 14, train accuracy: 78.0000%
Epoch 15, train accuracy: 84.0000%
Epoch 16, train accuracy: 92.0000%
Epoch 17, train accuracy: 84.0000%
Epoch 18, train accuracy: 78.0000%
Epoch 19, train accuracy: 78.0000%
Epoch 20, train accuracy: 92.0000%
Epoch 21, train accuracy: 88.0000%
Epoch 22, train accuracy: 78.0000%
Epoch 23, train accuracy: 84.0000%
Epoch 24, train accuracy: 92.0000%
Epoch 25, train accuracy: 82.0000%
Epoch 26, train accuracy: 82.0000%
Epoch 27, train accuracy: 90.0000%
Epoch 28, tr

Epoch 230, train accuracy: 98.0000%
Epoch 231, train accuracy: 90.0000%
Epoch 232, train accuracy: 100.0000%
Epoch 233, train accuracy: 98.0000%
Epoch 234, train accuracy: 96.0000%
Epoch 235, train accuracy: 98.0000%
Epoch 236, train accuracy: 94.0000%
Epoch 237, train accuracy: 94.0000%
Epoch 238, train accuracy: 100.0000%
Epoch 239, train accuracy: 100.0000%
Epoch 240, train accuracy: 96.0000%
Epoch 241, train accuracy: 96.0000%
Epoch 242, train accuracy: 94.0000%
Epoch 243, train accuracy: 98.0000%
Epoch 244, train accuracy: 94.0000%
Epoch 245, train accuracy: 98.0000%
Epoch 246, train accuracy: 96.0000%
Epoch 247, train accuracy: 98.0000%
Epoch 248, train accuracy: 98.0000%
Epoch 249, train accuracy: 98.0000%
Epoch 250, train accuracy: 100.0000%
Epoch 251, train accuracy: 98.0000%
Epoch 252, train accuracy: 94.0000%
Epoch 253, train accuracy: 92.0000%
Epoch 254, train accuracy: 100.0000%
Epoch 255, train accuracy: 96.0000%
Epoch 256, train accuracy: 98.0000%
Epoch 257, train accura

0.93714285

In [12]:
import tensorflow as tf

X_train_reshaped = np.reshape(X_train,(len(X_train), -1))
X_valid_reshaped = np.reshape(X_valid,(len(X_valid), -1))
X_test_reshaped = np.reshape(X_test,(len(X_test), -1))

def leaky_relu(alpha=0.01):
		def parametrized_leaky_relu(z, name=None):
			return tf.maximum(alpha * z, z, name=name)
		return parametrized_leaky_relu

params = {'n_hidden_layers': [0],
	'n_neurons' : [500],
	'optimizer_class' : [tf.train.AdagradOptimizer],
	'learning_rate' : [0.01],
	'batch_size' : [100],
	'activation' : [leaky_relu()],
	'dropout_rate' : [0.1]
}

grid_search = GridSearchDNN(params, k_fold=5)
grid_search.fit(X_train_reshaped, y_train, X_valid_reshaped, y_valid)

Trining DNN with parameters: n_hidden_layers: 0, n_neurons: 500, optimizer_class: <class 'tensorflow.python.training.adagrad.AdagradOptimizer'>, learning_rate: 0, batch_size: 100, activation: <function leaky_relu.<locals>.parametrized_leaky_relu at 0x0000020C905CBAE8>, dropout_rate: 0, 
Training and testing fold 0
ranges:
2312 2890
shapes:
(2314, 960) (578, 960)
(2314,) (578,)
0	Validation loss: 2.399198	Best loss: 2.399198	Accuracy: 37.58%
1	Validation loss: 1.877256	Best loss: 1.877256	Accuracy: 53.42%
2	Validation loss: 1.624942	Best loss: 1.624942	Accuracy: 58.39%
3	Validation loss: 1.475251	Best loss: 1.475251	Accuracy: 63.04%
4	Validation loss: 1.374180	Best loss: 1.374180	Accuracy: 67.70%
5	Validation loss: 1.297859	Best loss: 1.297859	Accuracy: 68.94%
6	Validation loss: 1.235907	Best loss: 1.235907	Accuracy: 69.25%
7	Validation loss: 1.187818	Best loss: 1.187818	Accuracy: 70.50%
8	Validation loss: 1.136724	Best loss: 1.136724	Accuracy: 73.60%
9	Validation loss: 1.103092	Best lo

120	Validation loss: 0.640929	Best loss: 0.640929	Accuracy: 82.61%
121	Validation loss: 0.639153	Best loss: 0.639153	Accuracy: 82.92%
122	Validation loss: 0.639910	Best loss: 0.639153	Accuracy: 82.30%
123	Validation loss: 0.641778	Best loss: 0.639153	Accuracy: 82.92%
124	Validation loss: 0.637029	Best loss: 0.637029	Accuracy: 82.92%
125	Validation loss: 0.638483	Best loss: 0.637029	Accuracy: 82.61%
126	Validation loss: 0.637804	Best loss: 0.637029	Accuracy: 82.61%
127	Validation loss: 0.638283	Best loss: 0.637029	Accuracy: 83.23%
128	Validation loss: 0.637511	Best loss: 0.637029	Accuracy: 83.23%
129	Validation loss: 0.637636	Best loss: 0.637029	Accuracy: 82.92%
130	Validation loss: 0.634480	Best loss: 0.634480	Accuracy: 82.30%
131	Validation loss: 0.631189	Best loss: 0.631189	Accuracy: 82.61%
132	Validation loss: 0.632464	Best loss: 0.631189	Accuracy: 82.61%
133	Validation loss: 0.630798	Best loss: 0.630798	Accuracy: 82.92%
134	Validation loss: 0.631365	Best loss: 0.630798	Accuracy: 82

KeyboardInterrupt: 

In [10]:
import tensorflow as tf

def leaky_relu(alpha=0.01):
		def parametrized_leaky_relu(z, name=None):
			return tf.maximum(alpha * z, z, name=name)
		return parametrized_leaky_relu
    
"""
Best parameters (kfold = 5):

{'n_hidden_layers': 2, 'n_neurons': 500, 'optimizer_class': <class 'tensorflow.python.training.adam.AdamOptimizer'>, 
'learning_rate': 0.05, 'batch_size': 400, 
'activation': <function leaky_relu.<locals>.parametrized_leaky_relu at 0x00000203351C67B8>, 
'dropout_rate': 0.1, 'conv1': {'conv1_fmaps': 16, 'conv1_ksize': 5, 'conv1_stride': 1, 
'conv1_dropout': 0.3, 'conv1_activation': <function relu at 0x00000203335AC620>}, 'conv2': {'conv2_fmaps': 16, 
'conv2_ksize': 5, 'conv2_stride': 1, 'conv2_dropout': 0.2, 'conv2_activation': <function relu at 0x00000203335AC620>}, 
'architecture': 4, 'accuracy_rate': 0.9958441495895386}

"""

params = {'n_hidden_layers': [1,2,3,4,5,6,7,8,9,10],
	'n_neurons' : [50, 150, 250, 300, 350, 400, 450, 500],
	'optimizer_class' : [tf.train.AdamOptimizer, tf.train.AdagradOptimizer],
	'learning_rate' : [0.01, 0.05, 0.1, 0.025],
	'batch_size' : [200, 250, 300, 400, 600],
	'activation' : [leaky_relu(), tf.nn.elu],
	'dropout_rate' : [0.000001, 0.1, 0.2, 0.3, 0.4],
	'conv1' : [{'conv1_fmaps':16, 'conv1_ksize':5, 'conv1_stride':1, 'conv1_dropout':0.1, 'conv1_activation':tf.nn.elu},
               {'conv1_fmaps':16, 'conv1_ksize':5, 'conv1_stride':1, 'conv1_dropout':0.2, 'conv1_activation':tf.nn.relu},
               {'conv1_fmaps':16, 'conv1_ksize':5, 'conv1_stride':1, 'conv1_dropout':0.3, 'conv1_activation':tf.nn.relu},
               {'conv1_fmaps':16, 'conv1_ksize':5, 'conv1_stride':1, 'conv1_dropout':0.1, 'conv1_activation':tf.nn.elu}],
	'conv2' : [{'conv2_fmaps':16, 'conv2_ksize':5, 'conv2_stride':1, 'conv2_dropout':0.1, 'conv2_activation':tf.nn.elu},
               {'conv2_fmaps':16, 'conv2_ksize':5, 'conv2_stride':1, 'conv2_dropout':0.2, 'conv2_activation':tf.nn.relu},
               {'conv2_fmaps':16, 'conv2_ksize':5, 'conv2_stride':1, 'conv2_dropout':0.3, 'conv2_activation':tf.nn.relu},
               {'conv2_fmaps':16, 'conv2_ksize':5, 'conv2_stride':1, 'conv2_dropout':0.1, 'conv2_activation':tf.nn.elu}],
    'architecture' : [1, 2, 3, 4]
}
rnd_search = RandomSearchCNN(params, k_fold=1, num_random_combinations=300)
rnd_search.fit(X_train, y_train, X_valid, y_valid)

testing 300 combinations.
Trining CNN with parameters: n_hidden_layers: 3, n_neurons: 50, optimizer_class: <class 'tensorflow.python.training.adagrad.AdagradOptimizer'>, learning_rate: 0, dropout_rate: 0, batch_size: 400, activation: <function elu at 0x0000023709B6E730>, conv1: {'conv1_fmaps': 16, 'conv1_ksize': 5, 'conv1_stride': 1, 'conv1_dropout': 0.2, 'conv1_activation': <function relu at 0x0000023709B77730>}, conv2: {'conv2_fmaps': 16, 'conv2_ksize': 5, 'conv2_stride': 1, 'conv2_dropout': 0.3, 'conv2_activation': <function relu at 0x0000023709B77730>} .architecture: 4, 
shapes:
(2185, 12, 80, 1) (385, 12, 80, 1)
(2185,) (385,)
***********
(2185, 12, 80, 1)
(2185,)
(643, 12, 80, 1)
(643,)
Epoch 0, train accuracy: 7.3227%, valid. accuracy: 6.5319%, valid. best loss: 4.078429
Epoch 1, train accuracy: 15.3318%, valid. accuracy: 12.9082%, valid. best loss: 3.673108
Epoch 2, train accuracy: 22.1968%, valid. accuracy: 17.5739%, valid. best loss: 3.434656
Epoch 3, train accuracy: 28.6041%

Epoch 84, train accuracy: 92.4485%, valid. accuracy: 83.3593%, valid. best loss: 0.540728
Epoch 85, train accuracy: 94.2792%, valid. accuracy: 83.0482%, valid. best loss: 0.540728
Epoch 86, train accuracy: 92.6773%, valid. accuracy: 83.6703%, valid. best loss: 0.540728
Epoch 87, train accuracy: 88.7872%, valid. accuracy: 83.3593%, valid. best loss: 0.540728
Epoch 88, train accuracy: 92.9062%, valid. accuracy: 84.1369%, valid. best loss: 0.540728
Epoch 89, train accuracy: 90.3890%, valid. accuracy: 83.3593%, valid. best loss: 0.539972
Epoch 90, train accuracy: 91.9908%, valid. accuracy: 84.4479%, valid. best loss: 0.539972
Epoch 91, train accuracy: 93.3638%, valid. accuracy: 84.1369%, valid. best loss: 0.539972
Epoch 92, train accuracy: 89.4737%, valid. accuracy: 84.2924%, valid. best loss: 0.539972
Epoch 93, train accuracy: 94.5080%, valid. accuracy: 84.4479%, valid. best loss: 0.536868
Epoch 94, train accuracy: 94.9657%, valid. accuracy: 83.0482%, valid. best loss: 0.536868
Epoch 95, 

Epoch 176, train accuracy: 97.0252%, valid. accuracy: 85.6921%, valid. best loss: 0.461288
Epoch 177, train accuracy: 96.5675%, valid. accuracy: 85.5366%, valid. best loss: 0.461288
Epoch 178, train accuracy: 97.2540%, valid. accuracy: 84.6034%, valid. best loss: 0.461288
Epoch 179, train accuracy: 98.3982%, valid. accuracy: 85.5366%, valid. best loss: 0.461288
Epoch 180, train accuracy: 97.4828%, valid. accuracy: 86.4697%, valid. best loss: 0.461288
Epoch 181, train accuracy: 96.3387%, valid. accuracy: 85.0700%, valid. best loss: 0.461288
Epoch 182, train accuracy: 96.5675%, valid. accuracy: 85.8476%, valid. best loss: 0.461288
Epoch 183, train accuracy: 96.1098%, valid. accuracy: 85.8476%, valid. best loss: 0.461288
Epoch 184, train accuracy: 96.3387%, valid. accuracy: 85.0700%, valid. best loss: 0.461288
Epoch 185, train accuracy: 97.2540%, valid. accuracy: 85.8476%, valid. best loss: 0.461288
Epoch 186, train accuracy: 96.3387%, valid. accuracy: 85.8476%, valid. best loss: 0.461288

Epoch 268, train accuracy: 98.1693%, valid. accuracy: 86.1586%, valid. best loss: 0.441342
Epoch 269, train accuracy: 97.4828%, valid. accuracy: 86.3141%, valid. best loss: 0.441342
Epoch 270, train accuracy: 97.9405%, valid. accuracy: 86.3142%, valid. best loss: 0.441342
Epoch 271, train accuracy: 97.9405%, valid. accuracy: 86.3142%, valid. best loss: 0.441342
Epoch 272, train accuracy: 97.0252%, valid. accuracy: 86.6252%, valid. best loss: 0.441342
Early stopping!
Final accuracy on test set: 0.971429
******************************************
{'n_hidden_layers': 3, 'n_neurons': 50, 'optimizer_class': <class 'tensorflow.python.training.adagrad.AdagradOptimizer'>, 'learning_rate': 0.01, 'batch_size': 400, 'activation': <function elu at 0x0000023709B6E730>, 'dropout_rate': 0.4, 'conv1': {'conv1_fmaps': 16, 'conv1_ksize': 5, 'conv1_stride': 1, 'conv1_dropout': 0.2, 'conv1_activation': <function relu at 0x0000023709B77730>}, 'conv2': {'conv2_fmaps': 16, 'conv2_ksize': 5, 'conv2_stride': 1

Epoch 72, train accuracy: 97.4359%, valid. accuracy: 87.5583%, valid. best loss: 0.523468
Epoch 73, train accuracy: 97.8022%, valid. accuracy: 85.3810%, valid. best loss: 0.523468
Epoch 74, train accuracy: 98.1685%, valid. accuracy: 86.4697%, valid. best loss: 0.523468
Early stopping!
Final accuracy on test set: 0.94026
******************************************
{'n_hidden_layers': 10, 'n_neurons': 150, 'optimizer_class': <class 'tensorflow.python.training.adagrad.AdagradOptimizer'>, 'learning_rate': 0.1, 'batch_size': 250, 'activation': <function elu at 0x0000023709B6E730>, 'dropout_rate': 0.1, 'conv1': {'conv1_fmaps': 16, 'conv1_ksize': 5, 'conv1_stride': 1, 'conv1_dropout': 0.1, 'conv1_activation': <function elu at 0x0000023709B6E730>}, 'conv2': {'conv2_fmaps': 16, 'conv2_ksize': 5, 'conv2_stride': 1, 'conv2_dropout': 0.3, 'conv2_activation': <function relu at 0x0000023709B77730>}, 'architecture': 4, 'accuracy_rate': 0.94025975465774536}
******************************************
Tr

KeyboardInterrupt: 